# What makes a galaxy quenched in TNG?


In [ ]:
from scida import load
import pandas as pd
import numpy as np
# load your favorite cosmological simulation snapshot
#ds = load("/data/cbyrohl/cosmosTNG/areporuns/cosmosTNG-2_Dec23-3/output/snapdir_033")
ds = load("TNG50-2_snapshot")
# dictionary of in-memory fields we want to run on
qdict = {k: ds.data["Subhalo"][k].compute().magnitude for k in ["SubhaloMass", "SubhaloVelDisp", "SubhaloGasMetallicity", "SubhaloBHMass"]}
qdict["SubhaloStellarMass"] = ds.data["Subhalo"]["SubhaloMassType"][:, 4].compute().magnitude
# most quantities should be log10 because of their dynamic range, replace them
log10quants = ["SubhaloMass", "SubhaloBHMass", "SubhaloGasMetallicity", "SubhaloStellarMass", "SubhaloVelDisp"]
for k in log10quants:
    q = qdict.pop(k)
    q[q==0] = 1e-99
    qdict[k+"_log10"] = np.log10(q)
df = pd.DataFrame.from_dict(qdict)

In [ ]:
sm = ds.data["Subhalo"]["SubhaloMassType"][:, 4].to("Msun").compute().magnitude
sm = np.where(sm > 0, sm, 1e-10)
ssfr = ds.data["Subhalo"]["SubhaloSFR"].to("Msun/yr").compute().magnitude / sm
X = df.iloc[:, :-1]

In [ ]:
# y: quenched or not
y = np.where(ssfr < 1e-11, 1, 0)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from interpret import set_visualize_provider

seed = 42
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [ ]:
from interpret.provider import InlineProvider, DashProvider
set_visualize_provider(DashProvider.from_address(('127.0.0.1', 7001)))
#set_visualize_provider(InlineProvider())

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

In [ ]:
from interpret import show
show(ebm.explain_global())